In [5]:
# maoyan movie top100
# 榜单规则：将猫眼电影库中的经典影片，按照评分和评分人数从高到低综合排序取前100名，每天上午10点更新。相关数据来源于“猫眼电影库”。
import requests
import re
import pymongo

In [6]:
# 常量
coll_name = 'maoyan_top100'
base_url = 'https://maoyan.com/board/4?offset=%d'  # 共10页，%s:range(0, 100, 10)
headers = {'User-Agent': 'Mozilla/5.0 (Windows; U; Windows NT 6.0; en-US; rv:1.9.1.6) Gecko/20091201 Firefox/3.5.6 GTB5'}

In [3]:
# 连接MongoDB
def conn_mongo():
    client = pymongo.MongoClient('localhost', 27017)
    db = client['spider']
    return client, db

In [13]:
# 采集
def movie_info(content):
    # ranking,img,title,movieId,star,releasetime,score
    movies = re.findall('board-index-(\d+).*?data-src="(.*?)".*?title="(.*?)".*?movieId:(\d+).*?class="star"\
>(.*?)</p>.*?releasetime"(.*?)</p>.*?class="score".*?integer">\
(.*?)</i>.*?fraction">(.*?)</i>', content, re.S)  # 换行用"\"，注意第二行不要留空格
    for movie in movies:
        if '(' in movie[5]:
            releaseCountry = re.search('\((.*)\)', movie[5]).group(1)
            releaseDate = re.search('上映时间：(.*?)\(', movie[5]).group(1)
        else:
            releaseCountry = None
            releaseDate = re.search('上映时间：(.*)', movie[5]).group(1)
        yield {
            'movieId': movie[3],
            'movieName': movie[2],
            'ranking': int(movie[0]),
            'score': float(movie[6] + movie[7]),
            'releaseDate': releaseDate,
            'releaseCountry': releaseCountry,
            'actors': re.sub('\s', '', movie[4]).split('：')[1].split(','),
            'img': movie[1]
        }

In [14]:
client, db = conn_mongo()
for offset in range(0, 100, 10):
    response = requests.get(base_url % offset, headers=headers, timeout=60)
    for i in movie_info(response.text):
        try:
            db[coll_name].insert_one(i)
        except:
            continue
client.close()